In [53]:
import numpy as np

In [54]:
def Gauss_Seidel(A, b, x, n):
    """
    Computes x vector from Ax=b linear matrix equation
    """
    
    L = np.tril(A)    # L is strictly lower triangluar matrix
    U = A - L         # U is upper triangulat matrix
    
    for i in range(n):
        x_new = np.dot(np.linalg.inv(L), b - np.dot(U, x))
        
        if np.all((abs(x_new-x)<=1e-8)):
            break
        #print str(i).zfill(3),
        
        x = x_new
        print(x)
    
    return x

In [55]:
A = np.array([[1.0, 2.0, 0, 0], [0, 3.0, 4.0, 0], [0, 0, -2, 1], [1, 2, 3, 4]])
b = [5, 6, 7, 8]
x = [1, 1, 1, 1]

In [56]:
def Viscosity_Air(T):
    """
    Correlation obtained from Kadoya K, Matsunaga N and Nagashima A - Viscosity and 
    Thermal Conductivity of Dry Air in the Gaseous Phases
    """
    
    Latitude = 11.3218072       # NITC location in degrees
    Longitude = 75.9344292
    Sea_level = 63              # meters
    
    Pressure = 101325*np.power((1 - 2.25577e-5*Sea_level), 5.25588)      # Pressure in Pa
    
    rho = Pressure / (287.058*T)       # density of air in kg m-3
    
    T_star = 132.5              # K
    rho_star = 314.3            # kg/m3
    H = 6.1609e-6               # Pa.s
    
    T_r = T/T_star
    rho_r = rho/rho_star
    
    A = np.array([0.128517, 2.60661, -1.0, -0.709661, 0.662534, -0.197846, 0.00770147 ])
    
    T_r_mat = np.array([T_r, np.power(T_r, 0.5), np.power(T_r, 0), np.power(T_r, -1), np.power(T_r, -2), np.power(T_r, -3), np.power(T_r, -4)])
    
    B = np.array([0.465601, 1.26469, -0.511425, 0.2746])
    
    rho_r_mat = np.array([rho_r, np.power(rho_r, 2), np.power(rho_r, 3), np.power(rho_r, 4)])
    
    viscosity = H * (np.dot(A, T_r_mat) + np.dot(B, rho_r_mat))
    
    return viscosity
    

In [57]:
Viscosity_Air(350)

2.0904818415092364e-05

In [58]:
def Thermal_Conductivity_Air_1(T):
    """
    Correlation obtained from Kadoya K, Matsunaga N and Nagashima A - Viscosity and 
    Thermal Conductivity of Dry Air in the Gaseous Phases
    """
    
    Latitude = 11.3218072       # NITC location in degrees
    Longitude = 75.9344292
    Sea_level = 63              # meters
    
    Pressure = 101325*np.power((1 - 2.25577e-5*Sea_level), 5.25588)      # Pressure in Pa
    

    rho = Pressure / (287.058*T)       # density of air in kg m-3

    T_star = 132.5              # K
    rho_star = 314.3            # kg/m3
    H = 25.9778e-3               # W m-1 K-1
    
    T_r = T/T_star
    rho_r = rho/rho_star
    
    A = np.array([0.239503, 0.00649768, 1, -1.92615, 2.00383, -1.07553, 0.229414 ])
    
    T_r_mat = np.array([T_r, np.power(T_r, 0.5), np.power(T_r, 0), np.power(T_r, -1), np.power(T_r, -2), np.power(T_r, -3), np.power(T_r, -4)])
    
    B = np.array([0.402287, 0.356603, -0.163159, 0.138059, -0.021725])
    
    rho_r_mat = np.array([rho_r, np.power(rho_r, 2), np.power(rho_r, 3), np.power(rho_r, 4), np.power(rho_r, 5)])
    
    conductivity = H * (np.dot(A, T_r_mat) + np.dot(B, rho_r_mat))
    
    return conductivity

In [59]:
Thermal_Conductivity_Air_1(1000)

0.067630678718804751

In [60]:
def Specific_Heat_Capacity_Air(T):
    
    #c_p = 1.9327e-10*np.power(T, 4) - 7.9999e-7*np.power(T, 3) + 1.1407e-3*np.power(T, 2) - 4.489e-1*T + 1.0575e3
    
    Tp = [250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 900, 1000, 1100, 1200, 1300, 1400]
    cp = [1003, 1005, 1008, 1013, 1020, 1029, 1040, 1051, 1063, 1075, 1087, 1099, 1121, 1142, 1155,1173, 1190, 1204]
    return np.interp(T, Tp, cp)

In [61]:
Specific_Heat_Capacity_Air(333)

1006.98

In [76]:
def Prantl_Air(T):
    
    Pr_air = Viscosity_Air(T)*Specific_Heat_Capacity_Air(T)/Thermal_Conductivity_Air_1(T)
    Tp = [175,
200,
225,
250,
275,
300,
325,
350,
375,
400,
450,
500,
550,
600,
650,
700,
750,
800,
850,
900,
950,
1000
]
    Pr = [0.744,
0.736,
0.728,
0.72,
0.713,
0.707,
0.701,
0.697,
0.692,
0.688,
0.684,
0.68,
0.68,
0.68,
0.682,
0.684,
0.687,
0.69,
0.693,
0.696,
0.699,
0.702
]
    return Pr_air, np.interp(T, Tp, Pr)

In [79]:
Prantl_Air(268)

(0.71591436189314972, 0.7149599999999999)

In [95]:
def Rayleigh_Air(T_g, T_a, dia_glass_ext):
    
    T = (T_g + T_a)/2
    
    Pressure = 101325*np.power((1 - 2.25577e-5*63), 5.25588)      # Pressure in Pa
    
    rho = Pressure / (287.058*T)        # density of air in kg m-3
   
    g = 9.81                            # accleration due to gravity in m s-2
    beta = 1/T
    
    dynamic_viscosity = Viscosity_Air(T)
    kinematic_viscosity = dynamic_viscosity / rho           #   m2 s-1
    
    thermal_diffusivity = Thermal_Conductivity_Air_1(T) / (rho*Specific_Heat_Capacity_Air(T))
    
    print (rho, dynamic_viscosity, Specific_Heat_Capacity_Air(T), Thermal_Conductivity_Air_1(T))
    
    T_s = T_g                   # glass envelope temperature in K
    T_inf = T_a                 # ambient environment temperature in K
    
    x = dia_glass_ext           # in m
    
    Ra = g*beta*(T_s - T_inf)*np.power(x,3)/(kinematic_viscosity*thermal_diffusivity)
    
    return Ra 

In [96]:
Rayleigh_Air(300, 280, 0.020)

1.20809968227 1.80815544933e-05 1004.6 0.0254850288641


17221.414181253032